# Investing with 'FACTOR INVESTING'

## What is the CAPM?

**Capital Asset Pricing Model** CAPM is a financial model to evaluate the expected return of an asset. It provides a framework for understanding the relationship between the asset's risk and its expected return.

- CAPM considers market risk to be the primary determinant of an asset's expected return.
- Market risk is measured by beta, which represents the asset's sensitivity to market movements.
- The CAPM formula is: E(X) = risk free rate + Beta * (Market Return - Risk Free Rate)
- The **risk free rate** represents the return on a risk free investment, for example; government bonds.
- The CAPM model helps investors determine if the asset value is undervalued or overvalued. 





